PART I

In [1]:
#importing all tools
import pandas as pd
import json
import xml.etree.ElementTree as et 
import html
from lxml import objectify
import urllib.request

In [2]:
#turning JSON file into Pandas dataframe (1 of 3 files)
JSONdf = pd.read_json (r'https://raw.githubusercontent.com/rleon207/DAV5400/master/Books_RLEON.json')

In [3]:
#checking to see how it looks in Pandas "as-is"
JSONdf

,RANDY'S 2ND FAVORITE BOOK TABLE
Author(s),Robert T. Kiyosaki
Book Title,RICH DAD POOR DAD
Page Count,287
Year Published,1997


In [4]:
#loading the HTML file into a Pandas DF (2 of 3)
HTMLdf = pd.read_html('https://raw.githubusercontent.com/rleon207/DAV5400/master/books_RLEON.html',skiprows=1)[0]

In [5]:
#checking to see how HTML data loads "as-is"
HTMLdf

,"""ONE UP ON WALL STREET""","Peter Lynch, John Rothchild",294,1989


This IS all of my data, however, let me clean this up a bit.

In [6]:
#I'm going to swap my data for the headers that belong
htmlheaders = ['Book Title', 'Author(s)', 'Page Count', "Year Published"]

In [7]:
HTMLdf.columns = htmlheaders

In [8]:
#sanity check
HTMLdf

,Book Title,Author(s),Page Count,Year Published


In [9]:
#using a dictionary to add values to my new headers
modifiedHTMLdf = HTMLdf.append({'Book Title' : 'ONE UP ON WALL STREET' , 'Author(s)' : 'Peter Lynch, John Rothchild', 'Page Count' : '294', 'Year Published' : '1989'} , ignore_index=True)

In [10]:
#sanity check
modifiedHTMLdf

,Book Title,Author(s),Page Count,Year Published
0,ONE UP ON WALL STREET,"Peter Lynch, John Rothchild",294,1989


In [11]:
#provided path to XML file (3 of 3)
path, headers = urllib.request.urlretrieve('https://raw.githubusercontent.com/rleon207/DAV5400/master/Books_RLEON.xml')
parsed = objectify.parse(open(path))
root = parsed.getroot()
root

<Element note at 0x11f0947c8>

In [12]:
#made empty dataset to store values
data = []

In [13]:
#used for loop to parse through XML data (as shown in lecture notes)
for elt in root:
    el_data = {}
    for child in elt.getchildren():
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [15]:
#sanity check
XMLdf = pd.DataFrame(data)
XMLdf

,Authors,BookTitle,PageCount,YearPublished
0,Charles Duhigg,SMARTER BETTER FASTER,285,2016


My JSON dataframe came out the most different than my HTML and XML dataframes. The JSON df had each index as a row, and the value in the subsequent column, while the HTML and XMLS had my data as a single row with each data point listed. The XML file came out as close as possible to the ideal dataframe in my opinion

PART II 

In [26]:
#importing the requests tool
import requests

In [47]:
#getting the amount of responses
url = 'https://api.nytimes.com/svc/mostpopular/v2/emailed/7.json?api-key=6JIc2EmFG5PY1TkLfT32tIVvqm7LRUHO'
resp = requests.get(url)
resp

<Response [200]>

In [48]:
#converting our data into a Python object
data2 = resp.json()
type(data2)

dict

In [49]:
#sanity check
data2

{'status': 'OK',
 'copyright': 'Copyright (c) 2019 The New York Times Company.  All Rights Reserved.',
 'num_results': 533,
 'results': [{'url': 'https://www.nytimes.com/2019/11/22/science/dogs-love-evolution.html',
   'adx_keywords': 'Animal Behavior;Dogs;Arizona State University;Wynne, Clive;Dog Is Love: Why and How Your Dog Loves You (Book);Books and Literature;your-feed-science',
   'subsection': '',
   'email_count': 1,
   'count_type': 'EMAILED',
   'column': None,
   'eta_id': 0,
   'section': 'Science',
   'id': 100000006823450,
   'asset_id': 100000006823450,
   'nytdsection': 'science',
   'byline': 'By JAMES GORMAN',
   'type': 'Article',
   'title': 'Dogs Can’t Help Falling in Love',
   'abstract': 'One researcher argues that a dog’s ability to bond has more to do with forming emotional attachments than being smart about what humans want.',
   'published_date': '2019-11-22',
   'source': 'The New York Times',
   'updated': '2019-11-26 15:11:25',
   'des_facet': ['ANIMAL BEH

In [74]:
#making result into a list
data3 = [data2]

In [78]:
# rows list initialization 
rows = [] 
  
# appending rows 
for data in data3: 
    data_row = data['results'] 
    for row in data_row: 
        row['adx_keywords'] 
        rows.append(row) 

#using dataframe
data4 = pd.DataFrame(rows) 
  

In [82]:
#sanity check
data4.head(n=5)

,abstract,adx_keywords,asset_id,byline,column,count_type,des_facet,email_count,eta_id,geo_facet,...,per_facet,published_date,section,source,subsection,title,type,updated,uri,url
0,One researcher argues that a dog’s ability to ...,Animal Behavior;Dogs;Arizona State University;...,100000006823450,By JAMES GORMAN,None,EMAILED,"[ANIMAL BEHAVIOR, DOGS, YOUR-FEED-SCIENCE]",1,0,,...,"[WYNNE, CLIVE]",2019-11-22,Science,The New York Times,,Dogs Can’t Help Falling in Love,Article,2019-11-26 15:11:25,nyt://article/8d1db2e8-3bb4-5c2b-893f-4c8c2210...,https://www.nytimes.com/2019/11/22/science/dog...
1,"In 1964, with “Seven Up!” Michael Apted stumbl...",Children and Childhood;Television;Documentary ...,100000006838494,By GIDEON LEWIS-KRAUS,None,EMAILED,[CHILDREN AND CHILDHOOD],2,0,[GREAT BRITAIN],...,"[APTED, MICHAEL]",2019-11-27,Magazine,The New York Times,,Does Who You Are at 7 Determine Who You Are at...,Article,2019-11-29 23:08:54,nyt://article/fd549a1d-cba8-5269-bf74-6faa71eb...,https://www.nytimes.com/2019/11/27/magazine/63...
2,"For 40 years, journalists chronicled the eccen...",India;Royal Families;News and News Media;Delhi...,100000006823322,,None,EMAILED,[NEWS AND NEWS MEDIA],3,0,"[INDIA, DELHI (INDIA), PAKISTAN]",...,"[CYRUS, PRINCE OF OUDH]",2019-11-22,World,The New York Times,asia pacific,The Jungle Prince of Delhi,Article,2019-11-28 01:00:24,nyt://article/03990cb5-2e65-5696-a899-fe839017...,https://www.nytimes.com/2019/11/22/world/asia/...
3,You are not just thinking with your brain.,"Brain;Vagus Nerve;Heart;Barrett, Lisa Feldman ...",100000006849074,By DAVID BROOKS,None,EMAILED,[BRAIN],4,0,,...,"[BARRETT, LISA FELDMAN (1963- ), PORGES, STEPH...",2019-11-28,Opinion,The New York Times,,The Wisdom Your Body Knows,Article,2019-11-30 21:57:27,nyt://article/3c4c308d-cf86-5598-9455-0348c83c...,https://www.nytimes.com/2019/11/28/opinion/bra...
4,The movie hits Netflix on Wednesday. Here’s a ...,Movies;The Irishman (Movie);I Heard You Paint ...,100000006831916,By MICHAEL WILSON,None,EMAILED,"[MOVIES, ORGANIZED CRIME, ORGANIZED LABOR]",5,0,,...,"[HOFFA, JAMES R, SHEERAN, FRANK]",2019-11-27,Movies,The New York Times,,The True Story of ‘The Irishman’: I Heard You ...,Article,2019-11-29 16:25:03,nyt://article/eb29ac92-fdfe-54e1-a632-0de564bd...,https://www.nytimes.com/2019/11/27/movies/the-...


I have successfully compiled all of the data from the API request into a neat DF that can be analyzed.

Thank you.

-R.L.